In [34]:
## Importing ecessary libraries and datasets

import torch
import torchvision
from torchvision.datasets import MNIST, CIFAR10, FashionMNIST
import torchvision.transforms as transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
## Downloading the data

mnist_train_data = MNIST(root='data/',train = True, download=True)
mnist_test_data =  MNIST(root='data/',train = False, download=True)

Fashion_train_data = FashionMNIST(root='data/',train = True, download=True)
Fashion_test_data =  FashionMNIST(root='data/',train = False, download=True)

cifar10_train_data = CIFAR10(root='data/',train = True, download=True)
cifar10_test_data =  CIFAR10(root='data/',train = False, download=True)


Extracting data/FashionMNIST\raw\train-images-idx3-ubyte.gz to data/FashionMNIST\raw




Extracting data/FashionMNIST\raw\train-labels-idx1-ubyte.gz to data/FashionMNIST\raw




Extracting data/FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data/FashionMNIST\raw




Extracting data/FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data/FashionMNIST\raw

Processing...
Done!



Extracting data/cifar-10-python.tar.gz to data/
Files already downloaded and verified


In [6]:
# Converting the downloaded data into tensor form to make it compatible with pytorch


mnist_train_data = MNIST(root='data/',train = True, download=True, transform=transforms.ToTensor())
mnist_test_data =  MNIST(root='data/',train = False, download=True, transform=transforms.ToTensor())

Fashion_train_data = FashionMNIST(root='data/',train = True, download=True, transform=transforms.ToTensor())
Fashion_test_data =  FashionMNIST(root='data/',train = False, download=True, transform=transforms.ToTensor())

cifar10_train_data = CIFAR10(root='data/',train = True, download=True, transform=transforms.ToTensor())
cifar10_test_data =  CIFAR10(root='data/',train = False, download=True, transform=transforms.ToTensor())

Files already downloaded and verified
Files already downloaded and verified


In [17]:
## Splitting the train data into Train and Validation Sets

from torch.utils.data import random_split

train_mnist, val_mnist = random_split(mnist_train_data, [50000, 10000])
print("Mnist train data: ", len(train_mnist), "Mnist Test data: ",len(val_mnist))
      
train_Fashion, val_Fashion = random_split(Fashion_train_data, [50000, 10000])
print("Fashion Mnist train data: ", len(train_Fashion), "Mnist Test data: ",len(val_Fashion))

train_cifar, val_cifar = random_split(cifar10_train_data, [40000, 10000])
print("Cifar 10 train data: ", len(train_cifar), "Mnist Test data: ",len(val_cifar))
      

Mnist train data:  50000 Mnist Test data:  10000
Fashion Mnist train data:  50000 Mnist Test data:  10000
Cifar 10 train data:  40000 Mnist Test data:  10000


In [18]:
## Loading the 3 datasets into batches of training and validation sets


from torch.utils.data import DataLoader

batch_size = 128

train_mnist_loader = DataLoader(train_mnist, batch_size, shuffle=True)
val_mnist_loader = DataLoader(val_mnist, batch_size)

train_Fashion_loader = DataLoader(train_Fashion, batch_size, shuffle=True)
val_Fashion_loader = DataLoader(val_Fashion, batch_size)

train_cifar_loader = DataLoader(train_cifar, batch_size, shuffle=True)
val_cifar_loader = DataLoader(val_cifar, batch_size)

# LOGISTIC REGRESSION MODEL

In [19]:
import torch.nn as nn

mnist_input_size = 28*28
fashionMNIST_input_size = 28*28
cifar10_input_size = 3* 32* 32

num_classes = 10


In [36]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [30]:
## Creating logistic regression Model for Fashion MNIST and MNIST

class MnistModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(mnist_input_size, num_classes)
        
    def forward(self, xb):
        xb = xb.reshape(-1, 784)
        out = self.linear(xb)
        return out
    
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss, 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['val_loss'], result['val_acc']))
        
        
    
model_mnist = MnistModel()
model_fashionMNIST = MnistModel()
print("MNIST & Fashion MNIST\nDimensions of Weight Matrix: ", model_mnist.linear.weight.shape,"\nDimensions of Bias Matrix", model_mnist.linear.bias.shape)

MNIST & Fashion MNIST
Dimensions of Weight Matrix:  torch.Size([10, 784]) 
Dimensions of Bias Matrix torch.Size([10])


In [31]:
## Creating Model for CIFAR 10

class CIFAR10_Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(cifar10_input_size, num_classes)
        
    def forward(self, xb):
        xb = xb.reshape(-1, cifar10_input_size)
        out = self.linear(xb)
        return out
    
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss, 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['val_loss'], result['val_acc']))
        
        

model_cifar10 = CIFAR10_Model()
print("CIFAR10\nDimensions of Weight Matrix: ", model_cifar10.linear.weight.shape,"\nDimensions of Bias Matrix", model_cifar10.linear.bias.shape)

CIFAR10
Dimensions of Weight Matrix:  torch.Size([10, 3072]) 
Dimensions of Bias Matrix torch.Size([10])


In [32]:
def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    optimizer = opt_func(model.parameters(), lr)
    history = [] # for recording epoch-wise results
    
    for epoch in range(epochs):
        
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result)
        history.append(result)

    return history


def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

In [37]:
## Training on MNIST Data

history1 = fit(5, 0.001, model_mnist, train_mnist_loader, val_mnist_loader)

Epoch [0], val_loss: 1.6637, val_acc: 0.7168
Epoch [1], val_loss: 1.4680, val_acc: 0.7502
Epoch [2], val_loss: 1.3199, val_acc: 0.7750
Epoch [3], val_loss: 1.2056, val_acc: 0.7908
Epoch [4], val_loss: 1.1156, val_acc: 0.8030


In [38]:
## Training on Fashion MNIST Data

history1 = fit(5, 0.001, model_mnist, train_Fashion_loader, val_Fashion_loader)

Epoch [0], val_loss: 1.8910, val_acc: 0.3233
Epoch [1], val_loss: 1.5369, val_acc: 0.5415
Epoch [2], val_loss: 1.3337, val_acc: 0.6105
Epoch [3], val_loss: 1.2043, val_acc: 0.6447
Epoch [4], val_loss: 1.1149, val_acc: 0.6685


In [40]:
## Training on CIFAR10 Data

history1 = fit(5, 0.001, model_cifar10, train_cifar_loader, val_cifar_loader)

Epoch [0], val_loss: 2.1456, val_acc: 0.2398
Epoch [1], val_loss: 2.0710, val_acc: 0.2720
Epoch [2], val_loss: 2.0249, val_acc: 0.2956
Epoch [3], val_loss: 1.9919, val_acc: 0.3123
Epoch [4], val_loss: 1.9691, val_acc: 0.3223
